In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path

In [2]:
df = pd.read_csv(Path('./data/20200817.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice]
0,8/12/2020 20:39:34,The Diary of a Teenage Girl,Bound,Junior,NaN,WWII (NO YANKS),No Theme/Free-for-all,SON: FRIEND OR FOE?,"SOMETHING OF A ""FLOWERS OF ALGERNON"" TYPE SITU...",WHODUNNIT?!
1,8/12/2020 20:43:56,Bound,Junior,They Came Together,The Diary of a Teenage Girl,SON: FRIEND OR FOE?,"SOMETHING OF A ""FLOWERS OF ALGERNON"" TYPE SITU...",WWII (NO YANKS),WHODUNNIT?!,No Theme/Free-for-all
2,8/15/2020 12:31:33,They Came Together,Bound,Junior,The Diary of a Teenage Girl,WHODUNNIT?!,SON: FRIEND OR FOE?,WWII (NO YANKS),No Theme/Free-for-all,NaN
3,8/12/2020 21:43:31,Junior,They Came Together,Bound,The Diary of a Teenage Girl,SON: FRIEND OR FOE?,"SOMETHING OF A ""FLOWERS OF ALGERNON"" TYPE SITU...",WWII (NO YANKS),WHODUNNIT?!,No Theme/Free-for-all
4,8/13/2020 6:54:33,Junior,Bound,They Came Together,The Diary of a Teenage Girl,WWII (NO YANKS),WHODUNNIT?!,SON: FRIEND OR FOE?,"SOMETHING OF A ""FLOWERS OF ALGERNON"" TYPE SITU...",NaN
5,8/13/2020 11:00:56,Junior,They Came Together,The Diary of a Teenage Girl,Bound,No Theme/Free-for-all,SON: FRIEND OR FOE?,WWII (NO YANKS),WHODUNNIT?!,"SOMETHING OF A ""FLOWERS OF ALGERNON"" TYPE SITU..."
6,8/15/2020 9:45:37,Bound,The Diary of a Teenage Girl,They Came Together,Junior,"SOMETHING OF A ""FLOWERS OF ALGERNON"" TYPE SITU...",SON: FRIEND OR FOE?,WHODUNNIT?!,WWII (NO YANKS),No Theme/Free-for-all


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Bound', 'Junior', 'They Came Together',
       'The Diary of a Teenage Girl'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Bound': <Candidate('Bound')>,
 'Junior': <Candidate('Junior')>,
 'They Came Together': <Candidate('They Came Together')>,
 'The Diary of a Teenage Girl': <Candidate('The Diary of a Teenage Girl')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(The Diary of a Teenage Girl, Bound, Junior)>,
 <Ballot(Bound, Junior, They Came Together, The Diary of a Teenage Girl)>,
 <Ballot(They Came Together, Bound, Junior, The Diary of a Teenage Girl)>,
 <Ballot(Junior, They Came Together, Bound, The Diary of a Teenage Girl)>,
 <Ballot(Junior, Bound, They Came Together, The Diary of a Teenage Girl)>,
 <Ballot(Junior, They Came Together, The Diary of a Teenage Girl, Bound)>,
 <Ballot(Bound, The Diary of a Teenage Girl, They Came Together, Junior)>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                      Votes  Status
---------------------------  -------  --------
Junior                             3  Hopeful
Bound                              2  Hopeful
They Came Together                 1  Rejected
The Diary of a Teenage Girl        1  Rejected

FINAL RESULT
Candidate                      Votes  Status
---------------------------  -------  --------
Bound                              4  Elected
Junior                             3  Rejected
They Came Together                 0  Rejected
The Diary of a Teenage Girl        0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                                                                    Votes  Status
-------------------------------------------------------------------------  -------  --------
SON: FRIEND OR FOE?                                                              2  Hopeful
WWII (NO YANKS)                                                                  2  Hopeful
SOMETHING OF A "FLOWERS OF ALGERNON" TYPE SITUATION IF YOU CATCH MY DRIFT        1  Hopeful
WHODUNNIT?!                                                                      1  Hopeful
No Theme/Free-for-all                                                            1  Rejected

ROUND 2
Candidate                                                                    Votes  Status
-------------------------------------------------------------------------  -------  --------
SON: FRIEND OR FOE?                                                              3  Hopeful
WWII (NO YANKS)                                               